In [336]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from scipy.io import arff
from sklearn.utils import shuffle

## NOTE::! I got massive problem with that original arff file (converting 'class' column to numeric)
* there was massive problem in converting the 'class' column of the original arff file into numeric type. I kept getting "ValueError cannot parse string"
* the arff data originally loaded correctly into the pandas dataframe, but I was not able to convert that 'class' column into numeric no matter what tricks I tried, (to_numeric or from teacher's example about using df['class'].replace() )



## I decided to take the initiative into my own hands, so then I just opened the arff file in notepad and copypasted the data into excel, and saved the spinal health data as a csv into my own computer

### everything went more smooth after that...

## How to solve the assignment

1. first we need that logisticReg model, from data 
2. then we can use the model to help predicting (Y_pred and Y_truelabels)
3. then we can get accuracy data and maybe cross validation accuracy
4. then we can ask the user to input data and have those predictions
5. predict_proba() should give the the probability of disease and the probability of the complimentary event (probability of healthy)
6. model.predict() should give the end result (disease or healthy), 

7. basically, we should get similar results from predict_proba, and model.predict when we simply input one row of patient data (that data which we ask the user to input)
8. so that basically if we apply round() function into the probability of  disease, we should get same result as we would get from predict, I think

In [337]:
#path=r'C:\Users\Lauri\Documents\HealthTechnology Mathematics&Methods\vertebral_column_data\column_2C_weka.arff'
#data = arff.loadarff(path)
#df = pd.DataFrame(data[0])

path1=r'C:\Users\Lauri\Documents\HealthTechnology Mathematics&Methods\spinal_health.csv'
df=pd.read_csv(path1, decimal='.',sep=',')

print(df.isnull().values.any()) #check missing values
print(df.dtypes)

False
pelvic_incidence            float64
pelvic_tilt                 float64
lumbar_lordosis_angle       float64
sacral_slope                float64
pelvic_radius               float64
degree_spondylolisthesis    float64
class                        object
dtype: object


In [338]:
df.describe()


,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496653,17.542822,51.930930,42.953831,117.920655,26.296694
std,17.236520,10.008330,18.554064,13.423102,13.317377,37.559027
min,26.147921,-6.554948,14.000000,13.366931,70.082575,-11.058179
25%,46.430294,10.667069,37.000000,33.347122,110.709196,1.603727
50%,58.691038,16.357689,49.562398,42.404912,118.268178,11.767934
75%,72.877696,22.120395,63.000000,52.695888,125.467674,41.287352
max,129.834041,49.431864,125.742385,121.429566,163.071041,418.543082


In [339]:
df.tail()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,Normal
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,Normal
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,Normal
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,Normal
309,33.841641,5.073991,36.641233,28.767649,123.945244,-0.199249,Normal


In [340]:
df.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal


In [341]:
#we have to change the output column into float64
# re-encode class column

# This is the part that I couldn't get working with the original arff file, 
# but now it works of course nicely with csv
df['class'].replace(['Normal', 'Abnormal'], [0.0,1.0], inplace=True)

#df=df.apply(pd.to_numeric)


print(df['class'][307])
print(df['class'][0])
print(df['class'].isnull().values.any())




0.0
1.0
False


In [342]:
df.tail()


,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,0.0
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,0.0
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,0.0
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,0.0
309,33.841641,5.073991,36.641233,28.767649,123.945244,-0.199249,0.0


In [343]:
df.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,1.0
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,1.0
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,1.0
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,1.0
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,1.0


In [344]:
# I think that the datapoints dont technically have to be shuffled, but I dont think it hurts either
# we will most likely use same trainingset only

# possibly we will benefit from this one shuffle, if we later on perform 
# k-fold crossval (without shuffling anymore inside that sklearn function)

df= shuffle(df)
df.reset_index(drop=True, inplace=True)
X = df.iloc[:,:6]
Y = df.iloc[:,6]



In [345]:
X #just printout to check dimensions

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
0,57.300227,24.188885,47.000000,33.111342,116.806587,5.766947
1,63.772391,12.763385,65.360524,51.009006,89.822741,55.995454
2,48.332638,22.227784,36.181993,26.104854,117.384625,6.481709
3,85.643787,42.689195,78.750664,42.954592,105.144076,42.887426
4,59.785265,17.879323,59.206461,41.905942,119.319111,22.123869
5,51.079833,14.209935,35.951229,36.869898,115.803711,6.905090
6,57.286945,15.149350,64.000000,42.137595,116.735387,30.341203
7,115.923261,37.515436,76.800000,78.407825,104.698603,81.198927
8,91.468741,24.508177,84.620272,66.960564,117.307897,52.623047
9,65.013773,9.838262,57.735837,55.175511,94.738525,49.696955


In [346]:
Y #just printout to check dimensions

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      0.0
6      1.0
7      1.0
8      1.0
9      1.0
10     1.0
11     0.0
12     1.0
13     1.0
14     0.0
15     1.0
16     1.0
17     1.0
18     1.0
19     1.0
20     1.0
21     1.0
22     1.0
23     1.0
24     1.0
25     0.0
26     1.0
27     1.0
28     1.0
29     1.0
      ... 
280    0.0
281    1.0
282    1.0
283    1.0
284    0.0
285    0.0
286    1.0
287    1.0
288    1.0
289    0.0
290    0.0
291    0.0
292    1.0
293    1.0
294    1.0
295    1.0
296    1.0
297    1.0
298    0.0
299    1.0
300    1.0
301    0.0
302    0.0
303    1.0
304    1.0
305    1.0
306    1.0
307    0.0
308    1.0
309    0.0
Name: class, Length: 310, dtype: float64

In [347]:
logreg=LogisticRegression(solver='lbfgs') # specifying solver supresses the python warnings
logreg.fit(X,Y)
Y_pred = logreg.predict(X)
cm = confusion_matrix(Y, Y_pred)
accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])


## Accuracy from training set and basic results

In [348]:

print("Coefficients: ",logreg.coef_)
print("Intercept: ", logreg.intercept_)


print("Confusion matrix:\n",cm)
print("Accuracy calculated from the training set = %.3f" % (accuracy))
print(classification_report(Y, Y_pred, target_names=['no', 'yes']))


Coefficients:  [[-0.0070301   0.08258075 -0.01870319 -0.08961076 -0.1067678   0.16811009]]
Intercept:  [15.15499492]
Confusion matrix:
 [[ 78  22]
 [ 22 188]]
Accuracy calculated from the training set = 0.858
              precision    recall  f1-score   support

          no       0.78      0.78      0.78       100
         yes       0.90      0.90      0.90       210

   micro avg       0.86      0.86      0.86       310
   macro avg       0.84      0.84      0.84       310
weighted avg       0.86      0.86      0.86       310



## Cross-validation accuracy
* note, that we already shuffled the data once, so this is good
* if we didnt shuffle the data before K-folds then, we would have had all the diseased patients first, followed by the healthy patients

In [349]:
# cross-validate
# number of folds
k = 10
scores = cross_val_score(estimator=logreg,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
print("Accuracies from %d individual folds:" % k)
print(scores)
print("Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))

Accuracies from 10 individual folds:
[0.70967742 0.96774194 1.         0.90322581 0.77419355 0.93548387
 0.77419355 0.80645161 0.77419355 0.77419355]
Accuracy calculated using 10-fold cross validation = 0.842


## Finally, we can ask the user for the measurements, as input
* then, we can use predict_proba
* we can also fit the model into that new patient data
* round(predict_proba) should be same as model.predict()

In [350]:
inputdatalist=[]
questionslist=["pelvic_incidence","pelvic_tilt","lumbar_lordosis_angle","sacral_slope","pelvic_radius","degree_spondylolisthesis"]
j=0
keepAsking=True
while j<6:
    input_p = input("%d. Give %s: "  % ((j+1), questionslist[j]) )
    try:
        val = float(input_p)
        print("valid answer given\n")
        inputdatalist.append(input_p)
        j=j+1
    except ValueError: 
        print("invalid type, please try again!\n")
        
print(inputdatalist)




1. Give pelvic_incidence: 48
valid answer given

2. Give pelvic_tilt: 14
valid answer given

3. Give lumbar_lordosis_angle: 36
valid answer given

4. Give sacral_slope: 34
valid answer given

5. Give pelvic_radius: 118
valid answer given

6. Give degree_spondylolisthesis: -4
valid answer given

['48', '14', '36', '34', '118', '-4']


In [351]:
new_userdata = pd.Series(inputdatalist)
new_userdata = new_userdata.values.reshape(1,-1) # reshape into proper 2D array dimension, even though one sample only


res = logreg.predict_proba(new_userdata)
print(res)
# get probability of something 
# (from simple debug testing the 
# 1st column should be p(isHealthy)
# 2nd column should be p(isSick)
# you can debug-test it by inputting healthy peoples' values, and sick peoples' values to see what comes out
print(logreg.predict(new_userdata))# get the actual output (0 should be healthy, 1 should be sick)


[[0.7344738 0.2655262]]
[0.]
